In [ ]:
# Jasmin comp_p silence script
from pydub import AudioSegment

# Paths to the audio files directory and the partly silenced audio files output directory
#
# !!! MAKE SURE TO CHANGE THESE IF YOU INTEND TO RUN THE CODE !!!
audio_path = '/home/jovyan/evaluation/jasmin_comp_p/group5/'
out_path = '/home/jovyan/evaluation/jasmin_comp_p/5_sil/'

# Open the STM file
#
# !!! MAKE SURE TO CHANGE THIS TO THE PATH OF THE STM FILE !!!
with open('/home/jovyan/evaluation/jasmin_comp_p/group5.stm') as file:
    # Read the utterances
    lines = file.readlines()
    # Keeps track of the audio file we are currently processing
    audiofile = ''
    # Variable that stores the actual audio array
    audio = ''

    # Iterate thru the lines
    for line in lines:
        # Lines that are comments start with ;;, so we ignore them
        if line[0] == ';':
            continue
        # Split the line on whitespace (whitespace is used as a delimiter for the
        # columns of the STM file)
        segments = line.split(' ')
        # If the current utterance's file ID has changed
        if segments[0] != audiofile:
            # If there is an audio file currently being processed (if it's
            # not the start of the script)
            if audiofile != '':
                # Save the audio with silenced parts
                audio.export(out_path + audiofile + '.wav', format='wav')
            # Update the file name of the currently processed audio file
            audiofile = segments[0]
            # Load the audio to be partly silenced
            audio = AudioSegment.from_file(audio_path + audiofile + '.wav')
            # Get the 2 channels
            channels = audio.split_to_mono()
            # Process only the speaker channel (1st one)
            audio = channels[0]
        # This is a gap where the speaker does not talk
        if segments[2] == 'inter_segment_gap':
            start = float(segments[3]) * 1000
            end = float(segments[4]) * 1000
            # Create a silence as long as the inter_segment_gap
            silence = AudioSegment.silent(duration=end-start)
            # Modify the audio such that we add the silence above where inter_segment_gap was
            audio = audio[:int(start)] + silence + audio[int(end):]

    # Since we exit the loop before we are able to save the last audio file,
    # we make sure we save it here
    audio.export(out_path + audiofile + '.wav', format='wav')